# 🤖 AI Code Generation Assistant

## Objective
Generate code snippets, unit tests, documentation, and refactor suggestions using multiple LLMs.

## Features
- Streamed generation for all models (OpenRouter + Ollama)
- File upload support: `.py`, `.txt`, `.ipynb`
- Single-model selector UI with provider toggle
- Fully modular — each concern is its own cell and function
- Interactive Gradio UI


Author: Naheem Quadri

---
## Imports

In [140]:
#imports
import os
import json
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

import asyncio
import sys



---
## Client Setup
Reads `OPENROUTER_API_KEY` and optionally `OLLAMA_BASE_URL` from  `.env` file
and returns ready-to-use OpenAI-compatible clients for both providers.

In [141]:
def setup_clients():
    """Initialise and return (openrouter_client, ollama_client).

    Reads credentials from environment / .env file.
    Raises RuntimeError if the OpenRouter API key is missing.
    """
    load_dotenv(override=True)

    api_key = os.getenv('OPENROUTER_API_KEY')
    if not api_key:
        raise RuntimeError(
            "OPENROUTER_API_KEY not found. "
            "Add it to your .env file: OPENROUTER_API_KEY=sk-..."
        )

    openrouter_client = OpenAI(
        base_url='https://openrouter.ai/api/v1',
        api_key=api_key
    )

    ollama_url = os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434/v1')
    ollama_client = OpenAI(
        base_url=ollama_url,
        api_key='ollama'   # required by the library; ignored by Ollama
    )

    print(f" OpenRouter client ready.")
    print(f" Ollama client pointed at: {ollama_url}")
    return openrouter_client, ollama_client


openrouter, ollama = setup_clients()

 OpenRouter client ready.
 Ollama client pointed at: http://localhost:11434/v1


---
##File Reading
Handles `.py`, `.txt`, and `.ipynb` uploads.

In [142]:
def extract_notebook_cells(content: str) -> str:
    """Parse a .ipynb JSON string and return all cell sources as plain text.

    Each cell is prefixed with its type ([CODE CELL] / [MARKDOWN CELL])
    so the model understands the context. Empty cells are skipped.

    Args:
        content: Raw .ipynb file content as a string.

    Returns:
        All non-empty cell sources joined by double newlines.
    """
    nb = json.loads(content)
    extracted = []
    for cell in nb.get('cells', []):
        src = ''.join(cell.get('source', [])).strip()
        if src:
            cell_type = cell.get('cell_type', 'code').upper()
            extracted.append(f"# [{cell_type} CELL]\n{src}")
    return '\n\n'.join(extracted)


def read_uploaded_file(file_obj) -> str:
    """Read an uploaded file and return its contents as a string.

    Supported formats:
      - .py  / .txt  — returned as-is
      - .ipynb       — cells extracted and concatenated via extract_notebook_cells()

    Args:
        file_obj: Gradio file object (has a .name path attribute), or None.

    Returns:
        File contents as a plain string, or "" if no file was provided.
    """
    if file_obj is None:
        return ""

    path = file_obj if isinstance(file_obj, str) else file_obj.name

    with open(path, 'r', encoding='utf-8') as f:
        content = f.read()

    if path.endswith('.ipynb'):
        return extract_notebook_cells(content)

    return content

---
## Prompt Builder
Constructs the structured prompt sent to the model.
Always requests a JSON response with keys: `code`, `tests`, `docs`, `refactor`.

In [143]:
def build_prompt(task_description: str, uploaded_code: str) -> str:
    """Build the full user prompt from a task description and optional code."""
    code_section = (
        f"Input code / context:\n```\n{uploaded_code}\n```"
        if uploaded_code.strip()
        else "No input code provided."
    )

    return f"""Task:
{task_description}

{code_section}

"""

---
## Streaming Helpers
One generator function per provider.
Both yield the accumulated response text after each incoming token chunk.

In [144]:
SYSTEM_PROMPT = """You are an expert AI coding assistant.

When given a coding task, respond using exactly this Markdown structure — no deviations:

## 💻 Generated Code
```python
# your code here
```

## 🧪 Unit Tests
```python
# your pytest tests here
```

## 📖 Documentation
Write clear docstrings, inline comments, and a short usage example here.

## ♻️ Refactor Suggestions
Write any refactoring or optimisation suggestions here. If none, write: *No refactor suggestions.*

Rules:
- Always include all four sections, in order
- Use the exact section headers shown above
- Do not add any text before the first section or after the last
"""


def stream_openrouter(model: str, prompt: str, temperature: float = 0.0):
    """Stream a chat completion from OpenRouter.

    Args:
        model:       OpenRouter model ID (e.g. 'openai/gpt-4o-mini').
        prompt:      The user prompt to send.
        temperature: Sampling temperature (0 = deterministic).

    Yields:
        Accumulated response text after each token chunk.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": prompt}
    ]
    stream = openrouter.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        temperature=temperature
    )
    collected = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        collected += delta
        yield collected


def stream_ollama(model: str, prompt: str, temperature: float = 0.5):
    """Stream a chat completion from a local Ollama instance.

    Args:
        model:       Ollama model tag (e.g. 'mistral:7b').
        prompt:      The user prompt to send.
        temperature: Sampling temperature.

    Yields:
        Accumulated response text after each token chunk.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": prompt}
    ]
    stream = ollama.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        temperature=temperature
    )
    collected = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        collected += delta
        yield collected

---
## Output Formatting
Parses the raw JSON response and renders each section as clean, readable Markdown.
Falls back gracefully if the model returns malformed JSON.

In [145]:
def format_output(raw: str, model_name: str, source_label: str) -> str:
    """Prepend a model header to the raw Markdown response.

    Args:
        raw:          Raw Markdown text returned by the model.
        model_name:   Display name of the model used.
        source_label: Provider label e.g. '☁️ OpenRouter' or '🖥️ Ollama'.

    Returns:
        Complete Markdown string ready for gr.Markdown.
    """
    if not raw.strip():
        return (
            f"## 🤖 {model_name}\n> {source_label}\n\n---\n\n"
            f"> ⚠️ Model returned an empty response.\n\n"
            f"Possible causes:\n"
            f"- Model ID `{model_name}` is incorrect or unavailable\n"
            f"- The request was rejected (content policy / token limit)\n"
            f"- Ollama: model may not be pulled — run `ollama pull {model_name}`"
        )

    header = f"## 🤖 {model_name}\n> {source_label}\n\n---\n\n"
    return header + raw.strip()

---
## Orchestration
`generate_code` is the single entry point called by Gradio.
It wires together all the module functions in sequence and streams output progressively.

In [146]:
def generate_code(
    model_source: str,
    model_name: str,
    task_description: str,
    pasted_code: str,
    temperature: float
):
    """Orchestrate the full code-generation pipeline for a single model."""

   
    if not task_description.strip():
        yield "> ⚠️ Please enter a task description before generating."
        return

    uploaded_code = pasted_code or ""
   
    prompt = build_prompt(task_description, uploaded_code)

    source_label = "☁️ OpenRouter" if model_source == "OpenRouter" else "🖥️ Ollama"
    stream_fn    = stream_openrouter if model_source == "OpenRouter" else stream_ollama

    yield f"## 🤖 {model_name}\n> {source_label}\n\n---\n\n⏳ *Connecting to model...*"

    try:
        raw = ""
        chunk_count = 0
        for raw in stream_fn(model_name, prompt, temperature):
            chunk_count += 1
            yield (
                f"## 🤖 {model_name}\n> {source_label}\n\n---\n\n"
                f"⏳ *Generating... ({len(raw)} chars)*\n\n```json\n{raw}\n```"
            )

        if not raw.strip():
            yield (
                f"## 🤖 {model_name}\n> {source_label}\n\n---\n\n"
                f"> ⚠️ **Model returned an empty response.**\n\n"
                f"Possible causes:\n"
                f"- Model ID `{model_name}` is incorrect or unavailable\n"
                f"- The request was rejected (content policy / token limit)\n"
                f"- Ollama: model may not be pulled — run `ollama pull {model_name}`\n\n"
                f"Try a different model or check your OpenRouter credits."
            )
            return

    except Exception as e:
        yield (
            f"## 🤖 {model_name}\n> {source_label}\n\n---\n\n"
            f"> ❌ **Stream error:** `{e}`\n\n"
            f"Check that the model ID is correct and your API key is valid."
        )
        return

    
    yield format_output(raw, model_name, source_label)

---
## UI Helpers & Model Lists
Defines the available models and the Gradio callback that swaps the
dropdown list when the user changes provider.

In [147]:
# Model lists

OPENROUTER_MODELS = [
    'openai/gpt-oss-120b',
    'x-ai/grok-4',
    'qwen/qwen3.5-27b',
    'anthropic/claude-3.5-haiku',
]

OLLAMA_MODELS = [
    'mistral:7b',
    'llama3.2:3b',
    'phi4-mini:latest',
    'gemma3:270m',
]


def get_model_list(source: str) -> list:
    """Return the correct model list for the chosen provider.

    Args:
        source: 'OpenRouter' or 'Ollama'.

    Returns:
        List of model ID strings.
    """
    return OPENROUTER_MODELS if source == "OpenRouter" else OLLAMA_MODELS


def update_model_dropdown(source: str):
    """Gradio callback: swap the model dropdown when the provider radio changes.

    Args:
        source: Value from the model_source Radio widget.

    Returns:
        gr.Dropdown update object with new choices and default value.
    """
    choices = get_model_list(source)
    return gr.Dropdown(choices=choices, value=choices[0])

---
## Gradio UI & Launch
Assembles all components into the final interface and starts the Gradio server.

In [149]:
with gr.Blocks(title='AI Code Generation Assistant', theme=gr.themes.Soft()) as demo:

   
    gr.Markdown("# 🤖 AI Code Generation Assistant")
    gr.Markdown(
        "Select a provider and model, describe your task, then click **Generate**.\n\n"
        "Optionally upload a `.py`, `.txt`, or `.ipynb` file as context."
    )

    
    with gr.Group():
        gr.Markdown("### ⚙️ Model Selection")
        with gr.Row():
            model_source = gr.Radio(
                choices=["OpenRouter", "Ollama"],
                value="OpenRouter",
                label="🔌 Provider",
                info="OpenRouter = cloud APIs  |  Ollama = local models"
            )
            model_dropdown = gr.Dropdown(
                choices=OPENROUTER_MODELS,
                value=OPENROUTER_MODELS[0],
                label="🧠 Model",
                info="Updates automatically when you switch provider"
            )

    
    model_source.change(
        fn=update_model_dropdown,
        inputs=model_source,
        outputs=model_dropdown
    )

   
    with gr.Group():
        gr.Markdown("### 📝 Task")
        with gr.Row():
            task_input = gr.Textbox(
                label="Task Description",
                placeholder='e.g. "Write a Python function to normalise a CSV column"',
                lines=4,
                scale=3
            )
            code_input = gr.Code(
            label="📂 Paste Code (optional — .py / .txt / .ipynb content)",
            language="python",
            lines=10,
            scale=1
        )

   
    with gr.Accordion("⚙️ Advanced Settings", open=False):
        temperature = gr.Slider(
            minimum=0.0,
            maximum=1.0,
            step=0.05,
            value=0.2,
            label="🎨 Temperature",
            info="Lower = more deterministic  |  Higher = more creative"
        )

   
    generate_btn = gr.Button("🚀 Generate Code", variant="primary", size="lg")


    gr.Markdown("### 📄 Output")
    output_panel = gr.Markdown(
        value="*Output will appear here after generation.*"
    )


    generate_btn.click(
        fn=generate_code,
        inputs=[model_source, model_dropdown, task_input, code_input, temperature],
        outputs=[output_panel]
    )



demo.launch(ssr_mode=False)

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.
